# tensorflow 2.0: Notebook 2: Computer Vision with CNNs 

## 1. Introduction to this Notebook 

* In this notebook, we will use the **`mnist`** dataset -  to build on our knowledge. In particular, we will:
  * introduce **`Computer Vision`** 
  * introduce **`convolutional layers`** into our models 
  * introduce the concept of **`regularisation`**
  * introduce how to **`save`** and reuse our model 
* The image below sets out how this fits within our deep learning framework and exising knowledge

![alt text](https://github.com/DanRHowarth/Tensorflow-2.0/blob/master/Notebook%202%20-%20Summary_final.png?raw=true)

### 1.1  Load Libraries

In [ ]:
## importing as per previous notebook

# We are future proofing by importing modules that modify or replace exising modules that we may have used now 
from __future__ import absolute_import, division, print_function, unicode_literals

# import tensorflow and tf.keras
import tensorflow as tf
from tensorflow import keras

# import helper libraries
import numpy as np
import matplotlib.pyplot as plt

# let's print out the version we are using 
print(tf.__version__)

In [ ]:
## some additional imports for this notebook 
from tensorflow.keras import datasets, layers, models

### 1.2 Loading our Data

In [ ]:
# split our data 
(train_images, train_labels), (test_images, test_labels) = datasets.mnist.load_data()

In [ ]:
# lets have a quick look at our data 
plt.imshow(train_images[0])

In [ ]:
# and at our labels
np.unique(train_labels)

**What is the problem we are trying to solve?**
* As we can see, we have images of digits from 0 - 9, and labels from 0 - 9. We are trying to build a model that correctly classifies the digits in the image.

## 2. Data: Introduction to Computer Vision 

**What is Computer Vision?**
* Computer Vision is the field of how computers can gain understanding from images and videos. It includes tasks such as **`image recognition`** and **`object detection`**. Deep Learning is seen as the state of the art technology for solving computer vision problems. 

**Why is Deep Learning particularly good at it?**
* The layers within a deep learning model are good for identifying and modelling the different aspects of an image (such as edges, parts of faces, and other important parts of an image). The meaning that each layer extracts can be built up to form representations for lots of different image types that can then be classified.  
* In particular, **`convolutional layers`** are good at extracting representation from image data and they form the basis of deep learning models for image recognition. The ability to build larger and larger models that consist of these convolutional layers, and to train them with more and more data (thanks to increasing compute power), led to a leap forward in state of the art for computer vision. 

**How does it work?**
* Every image is represented by an array of numbers. You may have noticed that the **`shape`** of the images we are processing. This shape represents the number of pixels in an image, and each pixel has a numerical value. This numerical value maps to a colour value that is displayed. It is also what we use as input values to our model. 

In [ ]:
## lets start by looking at the shape of an image

## we can see that it is 28 x 28 pixels
train_images[0].shape

In [ ]:
## we can also see that these pixels are represented in an array of numbers 
train_images[0]

In [ ]:
# we need plt.imshow() - or another library such as OpenCV or PIL - to output an image from this array
plt.imshow(train_images[0])

**What do the array values mean?**
* Each value leads to a colour for the pixel that the array value represents. Actualy what colour is displayed depends somewhat on the number of colour channels the array has. We have only one channel present in this dataset. This is grayscale channel. Typically, we will see three channels for colour images, with each channel representing one of Red, Green, Blue. A value in one channel will display a different colour than a value in another channel. 
* Its worth noting here that there are typically 256 values (0 - 255) available in each channel, making a total combination of c.
* We will rescale the arrays to between 0 and 1. This needs to happen in order to maximise the success of the training. 

**What about images of different shapes?**
* The size of an image can and does vary. In this case, we have small image of 28 x 28 pixels (or 28, 28, 1) given we have one channel.
* One final thing to note is that Deep Learning models always require an array of the same size to be passed to it. This means that images which differ in size need to be preprocessed so that they are the same size before being passed to the model. 

In [ ]:
# we now need to reshape the data to add a colour channel 
train_images = train_images.reshape((60000, 28, 28, 1))
test_images = test_images.reshape((10000, 28, 28, 1))

In [ ]:
# we can view the new shape 
train_images.shape

In [ ]:
# and normalize the data 
train_images, test_images = train_images / 255.0, test_images / 255.0

**So far pointers considered**
* Images have to be fed into the model in the same shape each time. This requires pre-processing.
* Prior to feed images into a model, we can also change the image in certain ways to add noise and variety to the training data. This should mean that the model is more robust and better at generalizing to unseen data. 

## 3. Model Building 

### 3.1 Convolutional Models

To built the model we will take an image as an input and flattened it to a **1D** array. 

**How do we build a convolutional neural network?**
* A convolutional neural network (CNN) contains both dense and convolutional layers. The convolutional layers form the **`base`** of the model and extracts representation from the image. The dense layers form the **`head`** of the model and takes this represetation and maps it to our output classes
* A convolutional layer takes our image as it (subject to any preprocessing to get it in a standard shape or augmented to add noise and variety to the dataset) - that is, we do not need to flatten the image into a **`1D`** array. We flatten the array after our final convolutional layer and prior to passing our input to the dense layer.

**Why use a convolutional layer?**
* A convolution better encodes the key information in an image than other types of layers. Their application to computer vision resulted in a marked improvement in what was state of the art. That's why we use them. 

**What is a convolutional layer?**
* Simply, a convolutional layer is a layer that performs a mathematical operations known as convolutional on the input data.
* Each convolutional layer have a user-defined set of filters (or windows) that we pass over the image. We define the number and size of filters, although they are typically a 3 x 3 matrix. 
* This filter contains a set of weights that will be learned by the model and which are used to multiply the input values and return a new value in the layer's output. Its these filters that contain the learning of the convolutional layers of the model, whose weights will be updated as we train so that they are more and more able to extract key information from the image.
* The filter is applied to all the image channels as it passes over each pixel location such that it will look at a specific row and column index position and all the array values available at that index:
$$(row, column, :)$$ 

**So what does a convolutional layer return?**
* A convolutional layer returns an output array of the same (row, column) shape as the input array, but with one channel only. 
* It tends to be the case that convolutional layer is paired with a **`pooling layer`**, a pooling layer tries to extract the key information from the convolutional layer while typically halving its size. 
* The diagram below set this out. 





![alt text](https://github.com/DanRHowarth/Tensorflow-2.0/blob/master/Notebook%202%20Convolution%20and%20Pooling%20Layers.png?raw=true)

In [ ]:
## lets build our convolutional base
## we use the Sequential API but use .add() rather than passing the layers in as a list

# build model using sequential 
model = models.Sequential() 

In [ ]:
# start adding layers. input shape has been defined, including the channel value we added via reshape earlier
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
# max pooling layers
model.add(layers.MaxPooling2D((2,2)))
# this is then repeated to build 
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
# max pooling layers
model.add(layers.MaxPooling2D((2,2)))
# additional convolutional layer 
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

In [ ]:
# print model 
model.summary()

**How do we get to the parameter count?**
* The parameters of a convolutional layer are defined by:
$$((filter\ height\ \times filter\ width)\ +\ bias\ term)\ \times number\ of\ filters$$

* The bias term is a value of 1 so the number of parameters for the first convolutional layer is:
$$((3 \times 3)+ 1)\ \times 32 = 320$$

**What about the classificaiton layer?**
* As we said above, a convolutional based needs a classification layer to take the information extracted from an image and map it to output classes. 
* We take the final output shape of the Convolutional layer and flatten it to a 1D array. We then define our output layer, which in this instance is a layer of 10 with a softmax activation function. We have also added an additional layer to provide additional parameters between the flattened and output layer. 

In [ ]:
# flatten 
model.add(layers.Flatten())
# add a fully connected layer
model.add(layers.Dense(64, activation='relu'))
# add the output layer, a fully connected layer with a softmax activation
model.add(layers.Dense(10, activation='softmax'))

In [ ]:
# complete model summary
model.summary()

## 4. Training 

### 4.1 Validation Sets, Batch Sizes and Learning Rates

* A validation set is a dataset that is kept aside from the training dataset. Tpyically, at the end of each epoch, the trained model is passed the validation set in inference model, and the loss and other metrics recorded. The model is then trained again for another epoch, and at the end of this epoch is passed the validation set in inference mode. 
* This allows us to see during training how the model performs on unseen data and also whether the model is **`under- or over-fitting`**. 

**What is under and overfitting?**
* When we train our models, they can sometimes struggle to generalise well. This means that they do not perform well on unseen data. 
* A model that overfits will get better and better results (loss and metrics) on the training data and decreasing results on the validation set. This is because it is fitting too much to the specific characteristics of the training data, which may not be present in the unseen data.
* A model that underfits does not perform well on either the training or validation data. 
* By incluidng a validation set,  we can monitor how well the model performs, and if the performance on the training and validation sets diverge too much then we can start to conclude that something need to be changed. 

**What can we do to address this?**
* We can use **`regularization`** to help prevent overfitting.
* We can regularize our model and its training in a number of ways, and to some extent they all penalize actions that may cause the model to overfit to the trianing data:
  * When building a model, we add a **`dropout`** layer. This turns off a user defined number of outputs from a layer r during training and helps prevents the model becoming reliant on certain paths through the model.
  *  During training, **`weight regularization`**. This penalises large weights (our learned parameters) in the model. Larger weights will create a larger loss value that the mdoel will then use to update the weights. A model with fewer larger weights, i.e. with the learning more evenly spread across the nodes, will have a better chance of generalizing well.   

**What is a batch_size?**
* A batch size is the number of samples the model trains on before performing a backward pass (model update). The number of batches in an epoch is equal to:
$$ \frac{number\ of\ sample\ in\ a\ training\ set}{batch\ size}$$
* We can set the batch size and choose to see the model performance as trains per batch size (using a parameter in **`model.fit()`**). 

**What is the learning rate?**
* The learning rate controls the size of the update to the learnable parameters (weights and biases) during the backward pass, based on the loss of the model.
* It is a parameter of the optimizer, set at the **`model.compile()`** stage, and can be set by the user. The trick is to set the learning to update the weights sufficiently to change performance, but not update them too much so that the model weights swing between higher and lower values without settling on a path to the best model.
* There are various strategies for mitigating this problem that are worth investigating.

In [ ]:
## Lets compile the model again

model.compile(optimizer='adam',
              
              loss='sparse_categorical_crossentropy',
              
              metrics=['accuracy'])

In [ ]:
## and build a training loop

history = model.fit(train_images, train_labels, 
          # add batch size
          batch_size = 32,
          # epochs           
          epochs = 10,
          # and add a validation set 
          validation_split=0.2)

**Did we do any good?**
* 98% plus on the validation set seems pretty good!
* Let's plot the loss and accuracy and see what it shows us 

In [ ]:
# if we print out our history object we can see that it now includes validation values
history_dict = history.history
history_dict.keys()

In [ ]:
## we can pass this to a pandas dataframe

# we need to import pandas
import pandas as pd

# pass history data to dataframe object
history_df = pd.DataFrame(history_dict)

# and display it 
history_df

In [ ]:
# we can use plot functionality of pandas to quickly plot our results
history_df.plot(figsize=(8,5))
# tailor our plot. Show the grid
plt.grid(True)
# set the vertical range to [0 -1]
plt.gca().set_ylim(0, 1)
# display plot
plt.show()

### 4.4 Saving Models

**What do we mean by *saving a model*?**
* We can save our progress as we train our models. We can save our progress in two ways:
  * *during training*, so that our models are saved after each epoch (or, after an epoch that shows model improvement). 
  * *after training*, so that our model has completed its training before we save it
* Of course, we can opt to save the model both during *and* after training. 
* Using `tensorflow 2.0`, we can opt to save the model either manually, i.e. after the model has trained, or by using callbacks - i.e. incorporating saving into the training process. 
  
**What are we saving?**
* We can save the model weights only, the full model (including the weights and the architecture), and the optimizer state.
* Its useful to remember that when we are training a model, the parameters we are updating during the training process are the weights at each layer of the model. Our aim is to train on (data, labels) pairs that mean we can predict effectively on unseen data using the weights we have trained. So it is these weights that are saved, optionally along with the model architecture. 
* Optimizer state. We haven't focussed too much on optimizers, but remember that this is the way that the model weights are updated. The size of the update is set by the (user defined) **`learning rate`**. When we save a model we can therefore save the **`optimizer-state`**, meaning we can continue training a loaded model from the state it was in when the model was saved.  

**Why save a model?**
* So that we can reuse it later. This could be to deploy it and use it in inference mode, or to continue training from the point at which we stopped. 

**Once we have saved a model, how do we use it again?**
* Once we have saved our weights and/or model, we can restore the model in a couple of different ways. If we decide to save the weights only, we need to create an identical model to the one that was used to create our weights. If we saved both the model and weights, we can load this entire model.

#### 4.3.1 Saving and Loading Weights Only

In [ ]:
## lets go through the steps of saving the model weights only

# here we define a location for the weights to be saved 
model.save_weights('./checkpoints/my_checkpoint')

In [ ]:
# to load the weights we need to create a new instance of the same model architecture 
new_instance = model

In [ ]:
# then we can load the weights 
new_instance.load_weights('./checkpoints/my_checkpoint')

In [ ]:
new_instance.weights[0][0][0][0]

#### 4.3.2 Saving and Loading an entire model

In [ ]:
# this saves it to the HDF5 format
model.save('my_model.h5')

In [ ]:
# recreate the saved model, including weights and optimizer 
new_model = keras.models.load_model('my_model.h5')

In [ ]:
new_model.summary()

###So, what did we cover in this section?
* Validation sets
* Overfitting 
* Regularization
* Learning Rates, Batch Sizes
* Saving Models

## 5. Evaluation and Inference 

In [ ]:
## using the model we have just saved, evaluate it on the test set 
## if you are stuck, use some of the code from notebook 1
# 
test_loss, test_acc = model.evaluate(test_images, test_labels)

## if we had saved our weights or model and loaded them up, we would use the model
## variable name we had saved here

## 6. Further Reading

* Work through Chapter 15 - page 517 - of Python Machine Learning for another implementation of a Convolutional Neural Network
* Optionally, work through Chapter 14 for a deeper understanding of tensorflow 2.0
* There are a lot of good articles out there too. For example, [Cezanne Comacho](https://cezannec.github.io/Convolutional_Neural_Networks/), and [Chris Olah](https://colah.github.io/posts/2014-07-Conv-Nets-Modular/) all provide a good understanding of how convolution works.
* For the maths of convolution, have a look at [this paper](https://arxiv.org/pdf/1603.07285.pdf). 